In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64

In [3]:
# Start SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review_Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Load in data from S3 bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Automotive_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RAB23OVFNCXZQ|          1|            0|          0|   N|                Y|
|R3NORADVJO6IE6|          5|            0|          0|   N|                Y|
|R299F4SO98S5OO|          5|            1|          1|   N|                Y|
|R2DA9DOT03UW6I|          5|            2|          3|   N|                Y|
|R2OGCH681EQHU6|          5|            0|          0|   N|                Y|
| R2JMKIC16MHD7|          5|            2|          2|   N|                Y|
|R1DB5DA7CWWTI8|          5|            0|          0|   N|                Y|
|R1N8XWFDK4QACP|          5|            0|          0|   N|                Y|
|R19VFLX6MRU0CX|          5|            0|          0|   N|                Y|
| RLTD8YDGTP9PT|          1|            0|          0|   N|     

In [6]:
# filter for only columns with 20 or more total votes 
cleaned_vine_df = vine_df.filter('total_votes>=20')
cleaned_vine_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1T4FVSABO0IDP|5          |8            |31         |N   |Y                |
|R397VXR1GAK6C9|5          |21           |21         |N   |Y                |
|R2H6KB9RHS17GA|3          |59           |69         |N   |Y                |
|R3HAK0MUN9F5IT|1          |6            |28         |N   |Y                |
|R29UTZTOPUVRSV|5          |21           |21         |N   |Y                |
|RY9FLUD8VG6XD |4          |27           |32         |N   |Y                |
|R3Q5NAY8BGAKHI|5          |34           |40         |N   |Y                |
|RRVJ7TYDEK7EO |5          |30           |30         |N   |Y                |
|R2S4CHISQA2WDH|5          |17           |20         |N   |Y                |
|R1JNALJ77OULPH|4          |57           |65         |N   |Y    

In [7]:
# filter for only columns with mostly helpful votes
helpful_vine_df = cleaned_vine_df.filter("helpful_votes/total_votes>=0.5")
helpful_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R397VXR1GAK6C9|          5|           21|         21|   N|                Y|
|R2H6KB9RHS17GA|          3|           59|         69|   N|                Y|
|R29UTZTOPUVRSV|          5|           21|         21|   N|                Y|
| RY9FLUD8VG6XD|          4|           27|         32|   N|                Y|
|R3Q5NAY8BGAKHI|          5|           34|         40|   N|                Y|
| RRVJ7TYDEK7EO|          5|           30|         30|   N|                Y|
|R2S4CHISQA2WDH|          5|           17|         20|   N|                Y|
|R1JNALJ77OULPH|          4|           57|         65|   N|                Y|
|R13YNG792BCR77|          5|           19|         20|   N|                Y|
|R1RYG8MVOPK4Q2|          5|           18|         20|   N|     

In [8]:
# load in a sql function to use columns 
from pyspark.sql.functions import col

# Filter last DF to only columns with vine == "Y" (Paid)
helpful_vine_y_df = helpful_vine_df.filter(col("vine") == "Y")
helpful_vine_y_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REKN4PLM5174P|          4|          219|        225|   Y|                N|
| RQOG8IIJX7CKB|          4|           21|         27|   Y|                N|
|R33EJ9PSZR13CM|          4|           26|         30|   Y|                N|
| RD2BFV896F321|          5|           16|         20|   Y|                N|
|R37GSXLA11XF8N|          5|           30|         34|   Y|                N|
| RQBQYBM9VMSH4|          5|           19|         20|   Y|                N|
|R2YE4F9ESKFWEK|          3|           20|         23|   Y|                N|
|R1E4L7HHB8TBP9|          4|           80|         92|   Y|                N|
|R2C2WXDXMZL005|          2|           20|         25|   Y|                N|
|R1PPY4U0K03KTE|          5|           31|         38|   Y|     

In [9]:
# Filter last DF to only columns with vine == "N" (Unpaid)
helpful_vine_n_df = helpful_vine_df.filter(col("vine") == "N")
helpful_vine_n_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R397VXR1GAK6C9|          5|           21|         21|   N|                Y|
|R2H6KB9RHS17GA|          3|           59|         69|   N|                Y|
|R29UTZTOPUVRSV|          5|           21|         21|   N|                Y|
| RY9FLUD8VG6XD|          4|           27|         32|   N|                Y|
|R3Q5NAY8BGAKHI|          5|           34|         40|   N|                Y|
| RRVJ7TYDEK7EO|          5|           30|         30|   N|                Y|
|R2S4CHISQA2WDH|          5|           17|         20|   N|                Y|
|R1JNALJ77OULPH|          4|           57|         65|   N|                Y|
|R13YNG792BCR77|          5|           19|         20|   N|                Y|
|R1RYG8MVOPK4Q2|          5|           18|         20|   N|     

In [10]:
# Determine total number of reviews
helpful_vine_df.count()

24824

In [11]:
# Determine total number of paid reviews
helpful_vine_y_df.count()

82

In [12]:
# Determine total number of unpaid reviews
helpful_vine_n_df.count()

24742

In [13]:
# Determine number of 5-star reviews
helpful_vine_df.filter(col("star_rating") == 5).count()

12840

In [14]:
# Determine number of 5-star paid reviews
helpful_vine_y_df.filter(col("star_rating") == 5).count()

33

In [15]:
# Determine number of 5-star unpaid reviews
helpful_vine_n_df.filter(col("star_rating") == 5).count()

12807

In [18]:
# Determine percentage of 5-star paid reviews
a = helpful_vine_y_df.count()
b = helpful_vine_y_df.filter(col("star_rating") == 5).count()
percantage_paid = b/a * 100
print(str(round(percantage_paid,1)) + "% of paid reviews are 5-stars")

40.2% of paid reviews are 5-stars


In [19]:
# Determine percentage of 5-star unpaid reviews
c = helpful_vine_n_df.count()
d = helpful_vine_n_df.filter(col("star_rating") == 5).count()
percentage_unpaid = d/c * 100
print(str(round(percentage_unpaid,1)) + "% of unpaid reviews are 5-stars")

51.8% of unpaid reviews are 5-stars
